In [15]:
import os
import threading
import pandas as pd
from collections import Counter

csv_store_path = os.environ['GP_HIST_PATH']
#csv_store_path = os.environ['my_home_path'] + '\data\space-track-gp-hist-sample'
file_list = 'all_files.txt'
csv_store_path

'D:\\Datasets\\Space-Track gp_history'

In [2]:
all_counts = Counter()
lock = threading.Lock()

def task(file_path):
    try:
        df = pd.read_csv(file_path, compression='gzip', low_memory=False)
        df = df[(df.MEAN_MOTION > 11.25) & (df.ECCENTRICITY < 0.25)]
    except:
        raise Exception(f'Failed to open {file_path}') 
    
    with lock:
        all_counts += Counter(df.NORAD_CAT_ID.to_list())

In [13]:
# https://www.toptal.com/python/beginners-guide-to-concurrency-and-parallelism-in-python
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
from time import time

def multiproc():
    ts = time()
    files = [file[:-1] for file in open(file_list).readlines()]
    
    #with ProcessPoolExecutor() as executor:
    with ThreadPoolExecutor() as executor:
        executor.map(task, files)
    print('Took %s seconds', time() - ts)
    
multiproc()

Took %s seconds 4.409718990325928


In [14]:
all_counts

Counter()

In [7]:
[file[:-1] for file in open(file_list).readlines()]

['D:\\Datasets\\Space-Track gp_history\\000000001-000100186.csv.gz',
 'D:\\Datasets\\Space-Track gp_history\\000100187-000200206.csv.gz',
 'D:\\Datasets\\Space-Track gp_history\\000200207-000300225.csv.gz',
 'D:\\Datasets\\Space-Track gp_history\\000300226-000400247.csv.gz',
 'D:\\Datasets\\Space-Track gp_history\\000400248-000500273.csv.gz']

In [3]:
%%time
with open(file_list, 'r') as f:
    for file in f.readlines():
        file = file[:-1]
        x = threading.Thread(target=task, args=(file,))
        x.start()

Wall time: 1.99 ms


Exception in thread Thread-10:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\cmtle\lib\threading.py", line 932, in _bootstrap_inner
Exception in thread Thread-7:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\cmtle\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\Anaconda3\envs\cmtle\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-dfaeaf2d5df7>", line 12, in threaded_task
    self.run()
  File "C:\ProgramData\Anaconda3\envs\cmtle\lib\threading.py", line 870, in run
UnboundLocalError: local variable 'all_counts' referenced before assignment
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-dfaeaf2d5df7>", line 12, in threaded_task
UnboundLocalError: local variable 'all_counts' referenced before assignment
Exception in thread Thread-9:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\cmtle\lib\th

In [7]:
all_counts.items()

dict_items([])

In [ ]:
with open('threaded_output.txt', 'w+') as f:
    for k,v in all_counts.items():
        f.write(str(k) + ',' + str(v))